# High-Quality Japanese Translation Through Target-Language Prompting

## 1. Understanding the Challenge

Japanese translation presents unique challenges for AI models that typical prompt engineering struggles to address:

- Japanese is highly contextual - meaning depends heavily on who is speaking to whom, their relationship, and the situation
- Multiple levels of politeness must be chosen based on complex social factors
- Cultural nuances affect word choice and expression
- Key information is often implicit rather than explicit
- Natural Japanese often omits subjects and context that English requires

## 2. Critical Safety Consideration (⚠️ Read First)

Before we begin implementation, understand that for any important translations:
- Always verify critical information (numbers, dates, names)
- Back-translate important content to check for completeness
- Get native speaker review for high-stakes content

This guide shows you how to get good translations, but verification is your responsibility.

## Setup

First, install required packages:

In [ ]:
!pip install "mirascope[gemini]"

In [ ]:
from typing import Optional, Tuple
from dataclasses import dataclass
from mirascope.core import BasePrompt, prompt_template, gemini

# Basic error handling for API calls
class TranslationError(Exception):
    """Raised when translation fails"""
    pass

@dataclass
class TranslationResult:
    """Stores translation output and metadata"""
    translation: str
    critique: str
    improvements: str
    iteration: int

## 3. The Bootstrapping Process

Key insight: Write prompts in Japanese by having the model translate your English prompt first.

In [ ]:
ENGLISH_BASE_PROMPT = """
Please translate the following English text into natural Japanese.
Consider:
- Speaker and audience relationship
- Situation formality
- Cultural context

After translation, identify any issues with:
- Word choice
- Natural flow
- Politeness level
- Cultural fit

Text: {text}
Context: {context}
"""

@prompt_template("""
Translate this prompt into natural Japanese:

{prompt}
""")
class PromptBootstrapper(BasePrompt):
    prompt: str

async def get_japanese_prompt() -> str:
    """Bootstrap our Japanese prompt"""
    try:
        bootstrapper = PromptBootstrapper(prompt=ENGLISH_BASE_PROMPT)
        response = await bootstrapper.run_async(gemini.call("gemini-1.5-pro"))
        return response.content
    except Exception as e:
        raise TranslationError(f"Failed to bootstrap prompt: {str(e)}")

# Example output when run:
# 以下の英文を自然な日本語に翻訳してください。
# 考慮すべき点：
# - 話者と聞き手の関係
# - 状況の形式度
# - 文化的背景
# ...

## 4. Translation Process

Now we'll implement the core translation process. Key to success is providing good context.

Examples of good context:
```python
# Good context: specific and detailed
context = "Junior engineer (1 year experience) emailing department head about project delay, first direct contact"

# Bad context: too vague
context = "Business email to boss"
```

In [ ]:
@prompt_template("{japanese_prompt}")
class TranslationWithCritique(BasePrompt):
    japanese_prompt: str
    text: str
    context: str

def parse_response(response: str) -> Tuple[str, str, str]:
    """Parse model response into translation, critique, improvements"""
    # Basic implementation - in practice you'd want more robust parsing
    parts = response.split("\n\n")
    translation = parts[0] if len(parts) > 0 else ""
    critique = parts[1] if len(parts) > 1 else ""
    improvements = parts[2] if len(parts) > 2 else ""
    return translation, critique, improvements

async def translate_with_iterations(text: str, context: str, num_iterations: int = 5) -> list[TranslationResult]:
    """Perform iterative translation with critique"""
    try:
        japanese_prompt = await get_japanese_prompt()
        translator = TranslationWithCritique(
            japanese_prompt=japanese_prompt,
            text=text,
            context=context
        )
        
        results = []
        for i in range(num_iterations):
            response = await translator.run_async(gemini.call("gemini-1.5-pro"))
            trans, crit, improv = parse_response(response.content)
            result = TranslationResult(
                translation=trans,
                critique=crit,
                improvements=improv,
                iteration=i+1
            )
            results.append(result)
            print(f"\nIteration {i+1}:")
            print(f"Translation: {trans}")
            print(f"Critique: {crit}")
            if not improv:  # No more improvements suggested
                print("No further improvements suggested, but continuing for safety")
        
        return results
    except Exception as e:
        raise TranslationError(f"Translation failed: {str(e)}")

## 5. Complete Example

Let's see everything in action with a real example:

In [ ]:
async def demonstrate_full_process():
    # Real example with numbers and dates to verify
    text = "Could you please review the Q1 report (budget: $75,000) by next Friday? We need to discuss the timeline (January-March 2024)."
    context = "Junior analyst emailing department head, formal business setting, first direct interaction"
    
    print("Original Text:", text)
    print("Context:", context)
    
    results = await translate_with_iterations(text, context, num_iterations=3)
    
    # Example output when run:
    # Iteration 1:
    # Translation: 大変恐れ入りますが、来週金曜日までにQ1レポート（予算：75,000ドル）のご確認をお願いできますでしょうか？
    # スケジュール（2024年1月～3月）について協議させていただきたく存じます。
    # Critique: 初めてのやり取りにしては丁寧さが足りない...
    #
    # Iteration 2:
    # Translation: 誠に恐れ入りますが、来週金曜日までにQ1レポート（予算：75,000ドル）のご査収を賜りたく存じます。
    # また、スケジュール（2024年1月～3月）についてもご相談させていただきたく存じます。
    # Critique: 適切な敬語レベルに修正済み...

    # Print final version for verification
    print("\nFinal Translation:")
    print(results[-1].translation)
    
    print("\nVerification Checklist:")
    print("✓ Numbers preserved: $75,000")
    print("✓ Dates preserved: January-March 2024")
    print("✓ Key requests preserved: review report, discuss timeline")

await demonstrate_full_process()

## Best Practices Summary

1. Always provide detailed context
2. Run at least 3 iterations, even if the model suggests stopping
3. Verify all numbers, dates, and names
4. Use back-translation to check critical content
5. Get native speaker review for important documents